In [1]:
# %load_ext autoreload
# %autoreload 2

import sys
import os
import pandas as pd
from datetime import datetime, timezone, timedelta

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as config
from src.inference import get_feature_store


In [2]:
import pandas as pd

df = pd.read_parquet(r"C:\Users\VAISHAK\OneDrive\Desktop\citibikenyc\data\raw\rides_2023_01.parquet")

# Ensure 'started_at' is datetime
df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')

# If 'pickup_hour' is not already present
if 'pickup_hour' not in df.columns:
    df['pickup_hour'] = df['started_at'].dt.hour

fixed_hour = df["pickup_hour"].dropna().max()
print(f"📅 Using most recent available hour for predictions: {fixed_hour}")


📅 Using most recent available hour for predictions: 23


In [4]:
import hopsworks

# 🔐 Authenticate with Hopsworks
project = hopsworks.login()

# 🌟 Get the feature store
fs = project.get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=config.FEATURE_GROUP_MODEL_PREDICTION_VERSION
)

# 📦 Read and filter data
df = fg.read()
predictions = df[df["pickup_hour"] == fixed_hour]


2025-05-11 04:11:25,285 INFO: Initializing external client
2025-05-11 04:11:25,289 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 04:11:26,218 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1228962
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.40s) 


In [11]:
import hopsworks
import pandas as pd

# Define feature group details
FEATURE_GROUP_MODEL_PREDICTION = "citi_bike_prediction"
FEATURE_GROUP_MODEL_PREDICTION_VERSION = 2

# 🔐 Authenticate with Hopsworks
project = hopsworks.login()

# 🌟 Get the feature store
fs = project.get_feature_store()

# Verify feature group
fg = fs.get_feature_group(
    name=FEATURE_GROUP_MODEL_PREDICTION,
    version=FEATURE_GROUP_MODEL_PREDICTION_VERSION
)

if fg is None:
    print(f"Feature group {FEATURE_GROUP_MODEL_PREDICTION} (version {FEATURE_GROUP_MODEL_PREDICTION_VERSION}) not found")
    feature_groups = fs.get_feature_groups()
    for fg in feature_groups:
        print(f"Name: {fg.name}, Version: {fg.version}")
    raise ValueError("Feature group not found")

# 📦 Read data
df = fg.read()
print("DataFrame columns:", df.columns.tolist())  # Debug: Confirm columns
print("DataFrame head:", df.head())  # Debug: Inspect data
print("Unique pickup_hour values:", df["pickup_hour"].unique())  # Debug: Check available pickup_hour
print("DataFrame size:", df.shape)  # Debug: Check if DataFrame is empty

if df.empty:
    raise ValueError("Feature group contains no data")

# Filter data
fixed_hour = "2025-05-11 03:00:00"  # Matches a value from df["pickup_hour"].unique()
predictions = df[df["pickup_hour"] == fixed_hour]
print("Filtered predictions:", predictions)  # Debug: Check filtered data

if predictions.empty:
    print(f"No data found for pickup_hour = {fixed_hour}")
    print("Unique pickup_hour values:", df["pickup_hour"].unique())
    raise ValueError("No matching data for specified pickup_hour")

# Sort and select top locations
top_locations = predictions.sort_values("prediction", ascending=False)
print(top_locations[["pickup_hour", "prediction"]].head(10))

2025-05-11 04:19:36,064 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 04:19:36,078 INFO: Initializing external client
2025-05-11 04:19:36,080 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 04:19:37,877 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1228962
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.42s) 
DataFrame columns: ['pickup_hour', 'prediction']
DataFrame head:            pickup_hour  prediction
0  2025-05-11 03:00:00         100
1  2025-05-11 04:00:00         150
Unique pickup_hour values: ['2025-05-11 03:00:00' '2025-05-11 04:00:00']
DataFrame size: (2, 2)
Filtered predictions:            pickup_hour  prediction
0  2025-05-11 03:00:00         100
           pickup_hour  prediction
0  2025-05-11 03:00:00         100
